## Automating Email Sending

### Setup

In [1]:
#Importing required packages

from dotenv import load_dotenv
load_dotenv()
import os
from pydantic import SecretStr



In [2]:
#Creating required llm which is from google
from langchain_google_genai import ChatGoogleGenerativeAI

#LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",api_key=SecretStr(os.getenv("GOOGLE_API_KEY")))

#Testing
llm.invoke("Hello")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-eb18b45a-01cb-46dd-8676-0f036419518f-0', usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})

### Getting Gamil Tool Kit for Automation

In [4]:

#Initial Setup

#For this it required the authentication from the user to actually sent the email and have permission

#This authentication is stored in token.json file and allowing the users email id to be a test user.

#Getting Gmail Tool kit
from langchain_google_community import GmailToolkit

#for custumization
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials
)


#For getting gamail creadentials
credentials = get_gmail_credentials(
    scopes=["https://mail.google.com/"],
    client_secrets_file="credentials.json"

    #Token file is already generated
)


#Allowing API access
api_resource = build_resource_service(credentials=credentials)


#Creating a GmailToolKit
gmailToolKit = GmailToolkit(api_resource=api_resource)

### Setting Email Agent

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver


#Getting tools 
tools = gmailToolKit.get_tools()

#For Memory
memory = MemorySaver()


#Creating Gmail agent with access to tools and with memory
email_agent = create_react_agent(llm,tools,checkpointer=memory)


### Email sending Function

In [6]:
from langchain_core.messages import SystemMessage,HumanMessage

def query_email_agent(query):

    #Configuration for memory
    config = {"configurable":{"thread_id":"emailAgentMemory"}}

    #Getting events of agent to print
    events = email_agent.stream(
        {"messages":[
                    SystemMessage(content="You are helpful email assistent who helps user with thier emails which are provided by user in query"),
                    HumanMessage(content=f"Query is :{query}")]},
        stream_mode="values",
        config=config
    )


    #A Loop for printing agent progress
    for event in events:
        event["messages"][-1].pretty_print()

### Calling Email Agent

In [59]:
#Defining query 

text = """

Here is a professional cold email that combines marketing and sales elements:

Subject: Expert Project Consultant for SAP Business AI Solutions

Dear [Recipient's Name],

I hope this email finds you well. My name is Kamal, and I am reaching out from DRC Systems India Limited, a renowned service-based company with 12+ years of experience in development. We have a proven track record of delivering exceptional solutions that meet our clients' needs.

I came across your requirement for a Project Consultant with expertise in SAP Business AI solutions, specifically in Machine Learning, SAP BTP, Cloud Identity services, IaSIPS, TensorFlow, scikit-learn, ABAP, Java, Python, JavaScript, NodeJS, Jupyter Notebook, Database, and Web and Cloud Computing. Our team has extensive experience in these areas, and we believe we can assist you in improving your business results.

Our consultants have a strong background in machine learning and have successfully implemented projects that leverage the latest technology to drive business growth. We would be delighted to discuss how our expertise can benefit your organization.

To give you a glimpse of our capabilities, I'd like to share some of our notable projects:

* Our Machine Learning and Python expertise: https://example.com/ml-python-portfolio
* Our experience with WordPress development: https://example.com/wordpress-portfolio

We understand that finding the right partner for your project can be challenging. That's why we want to assure you that our team is committed to delivering high-quality solutions that meet your specific needs.

If you're interested in learning more about how we can help, I'd be happy to set up a call to discuss your project requirements in more detail.

Please feel free to contact me at info@drcsystems.com or reply to this email. I look forward to the opportunity to collaborate with you.

Best regards,

Kamal
Marketing Department
DRC Systems India Limited
"""

In [60]:
email = "nkchauhan9090@gmail.com"

query = f"Create an email from, \n\n {text} and make sure that the email takes full width on screen. Draft it to {email}"

In [61]:
#Calling agent
query_email_agent(query)

================================ Human Message =================================

Query is :Create an email from, 

 

Here is a professional cold email that combines marketing and sales elements:

Subject: Expert Project Consultant for SAP Business AI Solutions

Dear [Recipient's Name],

I hope this email finds you well. My name is Kamal, and I am reaching out from DRC Systems India Limited, a renowned service-based company with 12+ years of experience in development. We have a proven track record of delivering exceptional solutions that meet our clients' needs.

I came across your requirement for a Project Consultant with expertise in SAP Business AI solutions, specifically in Machine Learning, SAP BTP, Cloud Identity services, IaSIPS, TensorFlow, scikit-learn, ABAP, Java, Python, JavaScript, NodeJS, Jupyter Notebook, Database, and Web and Cloud Computing. Our team has extensive experience in these areas, and we believe we can assist you in improving your business results.

Our consu

In [63]:
query_email_agent("sent the last exact drafted email While making sure that it is properly formated text.")

================================ Human Message =================================

Query is :sent the last exact drafted email While making sure that it is properly formated text.
================================== Ai Message ==================================
Tool Calls:
  send_gmail_message (936c94b2-017a-4f7d-ad00-4b89c67f6448)
 Call ID: 936c94b2-017a-4f7d-ad00-4b89c67f6448
  Args:
    to: ['nkchauhan9090@gmail.com']
    subject: Expert Project Consultant for SAP Business AI Solutions
    message: Dear [Recipient's Name],

I hope this email finds you well.

My name is Kamal, and I am reaching out from DRC Systems India Limited, a renowned service-based company with 12+ years of experience in development. We have a proven track record of delivering exceptional solutions that meet our clients' needs.

I came across your requirement for a Project Consultant with expertise in SAP Business AI solutions, specifically in Machine Learning, SAP BTP, Cloud Identity services, IaSIPS, TensorFlow

In [12]:
query = "Delete all the drafted emails"

query_email_agent("What are the functionality of the tools that you have?")

================================ Human Message =================================

Query is :What are the functionality of the tools that you have?
================================== Ai Message ==================================

I can use the following tools:

1.  `create_gmail_draft`: This tool allows me to create a draft email with a specified message, subject, and recipients (To, CC, and BCC).
2.  `send_gmail_message`: This tool enables me to send email messages to specified recipients with a subject and message.
3.  `search_gmail`: I can use this tool to search for email messages or threads based on a query. The query can include terms like sender, recipient, subject, date ranges, and more. I can also specify the maximum number of results to return and the type of resource to search for (messages or threads).
4.  `get_gmail_message`: This tool allows me to fetch a specific email by its message ID, which I can obtain from a search result. It returns details such as the thread ID, sn

In [15]:
query_email_agent("Are you able to read emails without message ID?")

================================ Human Message =================================

Query is :Are you able to read emails without message ID?
================================== Ai Message ==================================

I can't read emails directly without a message ID. However, I can search for emails based on different criteria (sender, recipient, subject, keywords, date, etc.) using the `search_gmail` tool. If the search returns any results, I can then use the message IDs from those results to read the content of specific emails using the `get_gmail_message` tool. So, while I can't read them directly without an ID, I can find them using a search query and then read them using their IDs.


In [17]:
query_email_agent("Search email about SAP Buisiness in draf section and give me it's message id. After returning message ID read the content of each email and give theme as a response")

================================ Human Message =================================

Query is :Search email about SAP Buisiness in draf section and give me it's message id. After returning message ID read the content of each email and give theme as a response
================================== Ai Message ==================================
Tool Calls:
  search_gmail (7222a72b-c1e3-4e46-929c-f9d9ab277583)
 Call ID: 7222a72b-c1e3-4e46-929c-f9d9ab277583
  Args:
    resource: messages
    query: in:draft "SAP Business"
================================= Tool Message =================================
Name: search_gmail

[{"id": "195b230e35e85db3", "threadId": "195b230e35e85db3", "snippet": "Dear [Recipient&#39;s Name], I hope this email finds you well. My name is Kamal, and I am reaching out from DRC Systems India Limited, a renowned service-based company with 12+ years of experience in", "body": "Dear [Recipient's Name],\n\nI hope this email finds you well. My name is Kamal, and I am reaching o

In [18]:
query_email_agent("What are the message ID of the emails in draft section?")

================================ Human Message =================================

Query is :What are the message ID of the emails in draft section?
================================== Ai Message ==================================
Tool Calls:
  search_gmail (38b2e6a3-faaf-4cdf-bd4e-f7b2595e5c00)
 Call ID: 38b2e6a3-faaf-4cdf-bd4e-f7b2595e5c00
  Args:
    resource: messages
    query: in:draft
================================= Tool Message =================================
Name: search_gmail

[{"id": "195b230e35e85db3", "threadId": "195b230e35e85db3", "snippet": "Dear [Recipient&#39;s Name], I hope this email finds you well. My name is Kamal, and I am reaching out from DRC Systems India Limited, a renowned service-based company with 12+ years of experience in", "body": "Dear [Recipient's Name],\n\nI hope this email finds you well. My name is Kamal, and I am reaching out from DRC Systems India Limited, a renowned service-based company with 12+ years of experience in development. We have a p

In [20]:
query_email_agent("Delete this email with message id 195b230e35e85db3")

================================ Human Message =================================

Query is :Delete this email with message id 195b230e35e85db3
================================== Ai Message ==================================

I am sorry, I am unable to delete emails. I can only create, send, search, and read emails.


In [22]:
query_email_agent(f"sent all emails from draft section to {email}. But make sure they are currectly formated before sending")

================================ Human Message =================================

Query is :sent all emails from draft section to nkchauhan9090@gmail.com. But make sure they are currectly formated before sending
================================== Ai Message ==================================

I can send all the emails from the draft section to nkchauhan9090@gmail.com. However, I need you to define what you mean by "correctly formatted." I can only use the tools I have, which are:

*   Read the content of the emails.
*   Send the emails as they are currently formatted in the draft section.

If you want me to make specific changes to the content or formatting, please provide me with clear instructions on what needs to be changed before sending. For example, do you want me to:

*   Change the subject line?
*   Modify the body of the email?
*   Add or remove recipients?

Once you provide me with these instructions, I can proceed with sending the emails.


In [23]:
query_email_agent("What is the last conversation?")

================================ Human Message =================================

Query is :What is the last conversation?
================================== Ai Message ==================================

The last thing we did was I asked you to define what you mean by "correctly formatted" before sending all the emails from the draft section to nkchauhan9090@gmail.com. I explained that I need clear instructions on what changes you want me to make to the content or formatting before sending them.


In [24]:
query_email_agent("By correctly formated i meant that each important paragraphs should be in a new line and there should be text format like no one can identify that it is auto generated email.")

================================ Human Message =================================

Query is :By correctly formated i meant that each important paragraphs should be in a new line and there should be text format like no one can identify that it is auto generated email.
================================== Ai Message ==================================

I understand. I will try my best to reformat the emails before sending them to make them look less like auto-generated emails. I will add line breaks between paragraphs and try to make the text flow more naturally.

Please be aware that I have limited capabilities in terms of advanced text formatting. I can add line breaks and adjust the text flow, but I cannot perform more sophisticated formatting tasks.

Also, I need to read the content of each email in the draft folder, reformat it according to your instructions, and then send it to nkchauhan9090@gmail.com.
Tool Calls:
  search_gmail (20a15c5b-2c11-4f52-8f98-c23a564e837c)
 Call ID: 20a15c5b

In [25]:
temp = """Dear [Recipient's Name], I hope this email finds you well. My name is Kamal, and I am reaching out from DRC Systems India Limited, a renowned service-based company with 12+ years of experience in development. We have a proven track record of delivering exceptional solutions that meet our clients' needs. I came across your requirement for a Project Consultant with expertise in SAP Business AI solutions, specifically in Machine Learning, SAP BTP, Cloud Identity services, IaSIPS, TensorFlow, scikit-learn, ABAP, Java, Python, JavaScript, NodeJS, Jupyter Notebook, Database, and Web and Cloud Computing. Our team has extensive experience in these areas, and we believe we can assist you in improving your business results. Our consultants have a strong background in machine learning and have successfully implemented projects that leverage the latest technology to drive business growth. We would be delighted to discuss how our expertise can benefit your organization. To give you a glimpse of our capabilities, I'd like to share some of our notable projects: * Our Machine Learning and Python expertise: https://example.com/ml-python-portfolio * Our experience with WordPress development: https://example.com/wordpress-portfolio We understand that finding the right partner for your project can be challenging. That's why we want to assure you that our team is committed to delivering high-quality solutions that meet your specific needs. If you're interested in learning more about how we can help, I'd be happy to set up a call to discuss your project requirements in more detail. Please feel free to contact me at info@drcsystems.com or reply to this email. I look forward to the opportunity to collaborate with you. Best regards, Kamal Marketing Department DRC Systems India Limited"""


query = f"You are sending email like this. {temp} I want it to be a better like have new line for each paragraphs and all like that please do so.  "

In [26]:
query_email_agent(query)

================================ Human Message =================================

Query is :You are sending email like this. Dear [Recipient's Name], I hope this email finds you well. My name is Kamal, and I am reaching out from DRC Systems India Limited, a renowned service-based company with 12+ years of experience in development. We have a proven track record of delivering exceptional solutions that meet our clients' needs. I came across your requirement for a Project Consultant with expertise in SAP Business AI solutions, specifically in Machine Learning, SAP BTP, Cloud Identity services, IaSIPS, TensorFlow, scikit-learn, ABAP, Java, Python, JavaScript, NodeJS, Jupyter Notebook, Database, and Web and Cloud Computing. Our team has extensive experience in these areas, and we believe we can assist you in improving your business results. Our consultants have a strong background in machine learning and have successfully implemented projects that leverage the latest technology to drive bu